In [ ]:
DATE = "2022-02-19"
TASK = "sample-core-genome"
DIR = mkpath("$(homedir())/workspace/$(DATE)-$(TASK)")
cd(DIR)

In [ ]:
pkgs = [
    "Graphs",
    "MetaGraphs",
    "FileIO",
    "JLD2",
    "DataStructures",
    "BioSequences",
    "Random",
    "FASTX",
    "Revise",
    "StatsBase",
    "Statistics"
]

import Pkg
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $(basename(pkg))"))
end

import Mycelia

In [ ]:
function update_remaining_targets(current_walk::AbstractVector{T}, remaining_targets::AbstractSet{T}) where T <: BioSequences.AbstractMer
    # assess whether targets have been hit in the canonical space
    remaining_targets = setdiff(BioSequences.canonical.(remaining_targets), BioSequences.canonical.(current_walk))
    # blow back out into forward and reverse_complement space
    remaining_targets = Set{T}(vcat(remaining_targets, BioSequences.reverse_complement.(remaining_targets)))
    return remaining_targets
end

In [ ]:
function observe(records::AbstractVector{R};
                weights=ones(length(records)),
                N = length(records),
                outfile = "",
                error_rate = 0.0) where {R <: Union{FASTX.FASTA.Record, FASTX.FASTQ.Record}}
    if isempty(outfile)
        error("no file name supplied")
    end
    io = open(outfile, "w")
    fastx_io = FASTX.FASTA.Writer(io)
    for i in 1:N
        record = StatsBase.sample(records, StatsBase.weights(weights))
        new_seq = Mycelia.observe(FASTX.sequence(record), error_rate=error_rate)
        new_seq_id = Random.randstring(Int(ceil(log(length(new_seq) + 1))))
        new_seq_description = FASTX.identifier(record)
        observed_record = FASTX.FASTA.Record(new_seq_id, new_seq_description, new_seq)
        write(fastx_io, observed_record)
    end
    close(fastx_io)
    close(io)
    return outfile
end

In [ ]:
function assess_downstream_weight(graph, kmer)
    # here we look to see if walking forward or backward from the initial node gets us to heavier weight options
    score = 0
    for neighbor in BioSequences.neighbors(kmer)
        try
            score += MetaGraphs.get_prop(graph, graph[BioSequences.canonical(neighbor), :kmer], :count)
        catch
            continue
        end
    end
    return score
end

In [ ]:
# vertices should either be entire graph (by default) or a connected component
# if people want to work on just the connected component, let them induce a subgraph
function find_graph_core(graph; seed=rand(Int))
    
    Random.seed!(seed)
    
    T = typeof(MetaGraphs.get_prop(graph, 1, :kmer))
    
    # initialize points where coverage >= mean and degree as high as possible
    mean_depth_of_coverage = Statistics.mean([MetaGraphs.get_prop(graph, v, :count) for v in Graphs.vertices(graph)])
    
    selected_nodes = Set{T}([
                MetaGraphs.get_prop(graph, v, :kmer) for v in Graphs.vertices(graph) 
                    if (MetaGraphs.get_prop(graph, v, :count) >= mean_depth_of_coverage)])
    induced_subgraph, v_map = Graphs.induced_subgraph(graph, [graph[kmer, :kmer] for kmer in selected_nodes])
    MetaGraphs.set_indexing_prop!(induced_subgraph, :kmer)
    
    starting_kmer = first(selected_nodes)
    max_degree = 0
    for node in selected_nodes
        node_degree = Graphs.degree(induced_subgraph, induced_subgraph[node, :kmer])
        if node_degree > max_degree
            max_degree = node_degree
            starting_kmer = node
        end
    end
        
    current_walk = [starting_kmer]
    prior_walk_length = length(current_walk)
    remaining_targets = update_remaining_targets(current_walk, selected_nodes)
    done = isempty(remaining_targets)
    
    while !done
        # here we look to see if walking forward or backward from the current ends gets us to heavier weight options
        # we want to prioritize walks toward higher coverage nodes
        forward_score = assess_downstream_weight(induced_subgraph, last(current_walk))
        reverse_score = assess_downstream_weight(induced_subgraph, BioSequences.reverse_complement(first(current_walk)))
        if reverse_score > forward_score
            current_walk = reverse(BioSequences.reverse_complement.(current_walk))
        end
        
        forward_source = last(current_walk)
        forward_walk, forward_distance = Mycelia.dijkstra(induced_subgraph, forward_source, remaining_targets, search_strategy=:DFS)
        current_walk = vcat(current_walk, forward_walk[2:end])
        remaining_targets = update_remaining_targets(current_walk, remaining_targets)
        if isempty(remaining_targets)
            done = true
        else
            reverse_source = BioSequences.reverse_complement(first(current_walk))
            reverse_walk, reverse_distance = Mycelia.dijkstra(induced_subgraph, reverse_source, remaining_targets, search_strategy=:DFS)
            current_walk = vcat(reverse(BioSequences.reverse_complement.(reverse_walk))[1:end-1], current_walk)
            remaining_targets = update_remaining_targets(current_walk, remaining_targets)
            current_walk
        end
        failed_this_expansion = length(current_walk) == prior_walk_length
        prior_walk_length = length(current_walk)
        if isempty(remaining_targets)
            done = true
        elseif failed_this_expansion
            done = true
        end
    end
    return current_walk
end     

In [ ]:
# Create a reference sequence
L = 100
seed = 0
# seed = 1
# seed = 2
# seed = 3
# seed = 4
# seed = 5
# seed = 6
# seed = 7
# seed = 8
# seed = 9
record = Mycelia.random_fasta_record(seed=seed, L=L)

In [ ]:
error_rate = 0.1

In [ ]:
observations_file = "$(DIR)/N-$L.observations.fasta"
observe([record], N=L, outfile=observations_file, error_rate=error_rate)

In [ ]:
chosen_k = Mycelia.assess_kmer_saturation([observations_file]; outdir="$(observations_file).kmer-assessements")

In [ ]:
T = BioSequences.BigDNAMer{chosen_k}
# T = BioSequences.BigDNAMer{61}

In [ ]:
graph = Mycelia.fastx_to_kmer_graph(T, observations_file)

In [ ]:
Mycelia.graph_to_gfa(graph, "$(observations_file).gfa")

In [ ]:
FASTX.sequence(record)

In [ ]:
join(map(kmer -> graph[kmer, :kmer], BioSequences.canonical.(BioSequences.each(T, FASTX.sequence(record)))), ',')

In [ ]:
join(map(kmer -> string(graph[BioSequences.canonical(kmer), :kmer]) * (BioSequences.iscanonical(kmer.fw) ? '+' : '-'), collect(BioSequences.each(T, FASTX.sequence(record)))), ',')

In [ ]:
seed = 0
core_path = find_graph_core(graph, seed=seed)
reconstruted_sequence = Mycelia.kmer_path_to_sequence(core_path)
Mycelia.is_equivalent(reconstruted_sequence, FASTX.sequence(record))

In [ ]:
selected_vertices = map(canonical_kmer -> graph[canonical_kmer, :kmer], unique(BioSequences.canonical.(core_path)))
subgraph, vertex_map = Graphs.induced_subgraph(graph, selected_vertices)
Mycelia.plot_graph(subgraph)

In [ ]:
Mycelia.is_equivalent(reconstruted_sequence, FASTX.sequence(record))

In [ ]:
seed += 1
@show seed
core_path = find_graph_core(graph, seed=seed)
reconstruted_sequence = Mycelia.kmer_path_to_sequence(core_path)
Mycelia.is_equivalent(reconstruted_sequence, FASTX.sequence(record))